# 1.0 - Entrenamiento del Modelo de Diagnóstico Respiratorio

**Objetivo:** Cargar el dataset sintético, preprocesarlo, entrenar un `DecisionTreeClassifier` y guardar el modelo entrenado para su uso en producción.

## 1. Cargar Librerías y Datos

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import joblib
import os

# --- Rutas ---
DATASET_PATH = '../data/processed/dataset_respiratorio.csv'
MODEL_DIR = '../models/'
MODEL_PATH = os.path.join(MODEL_DIR, 'diagnostico_dt_model.pkl')

# Crear directorio si no existe
os.makedirs(MODEL_DIR, exist_ok=True)

# Cargar datos
df = pd.read_csv(DATASET_PATH)

print("Dataset cargado exitosamente.")
print(f"Shape: {df.shape}")
df.head()

## 2. Preprocesamiento de Datos

El `DecisionTreeClassifier` de scikit-learn requiere que todas las características de entrada sean numéricas. Por lo tanto, debemos convertir las columnas categóricas (como `tos` o `fumador`) a un formato numérico. Usaremos `OneHotEncoder` para esto.

In [ ]:
# Separar características (X) y etiquetas (y)
# Vamos a predecir tanto el diagnóstico como la gravedad.
# En una implementación más avanzada, se podrían entrenar dos modelos separados.
X = df.drop(['diagnostico', 'gravedad'], axis=1)
y_diag = df['diagnostico']
y_grav = df['gravedad']

# Identificar columnas categóricas y numéricas
categorical_features = X.select_dtypes(include=['object', 'bool']).columns
numerical_features = X.select_dtypes(include=np.number).columns

print("Columnas Categóricas:", categorical_features.tolist())
print("Columnas Numéricas:", numerical_features.tolist())

In [ ]:
# Crear el transformador de preprocesamiento
# OneHotEncoder para las categóricas, y 'passthrough' para las numéricas (ya están listas)
preprocessor = ColumnTransformer(
    transformers=[
        ('num', 'passthrough', numerical_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ])

## 3. Entrenamiento del Modelo de Árbol de Decisión

Entrenaremos un modelo para predecir el **diagnóstico**. Podríamos entrenar otro para la gravedad si fuera necesario.

In [ ]:
# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y_diag, test_size=0.2, random_state=42, stratify=y_diag)

# Crear el pipeline del modelo
model_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                               ('classifier', DecisionTreeClassifier(max_depth=5, random_state=42))])

# Entrenar el modelo
model_pipeline.fit(X_train, y_train)

print("Modelo entrenado exitosamente.")

## 4. Evaluación del Modelo

In [ ]:
# Realizar predicciones en el conjunto de prueba
y_pred = model_pipeline.predict(X_test)

# Evaluar el rendimiento
print("Accuracy:", accuracy_score(y_test, y_pred))
print("
Classification Report:")
print(classification_report(y_test, y_pred))
print("
Confusion Matrix:")
print(pd.DataFrame(confusion_matrix(y_test, y_pred), index=model_pipeline.classes_, columns=model_pipeline.classes_))

## 5. Guardar el Modelo Entrenado

Finalmente, guardamos el pipeline completo (preprocesador + clasificador) en un archivo `.pkl` para que pueda ser cargado y utilizado por nuestra aplicación Flask.

In [ ]:
# Guardar el pipeline del modelo
joblib.dump(model_pipeline, MODEL_PATH)

print(f"Modelo guardado en: {MODEL_PATH}")